In [79]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import datetime
import numpy as np

## Reading excel file


In [9]:
xls = pd.ExcelFile('data.xlsx')

## Reading individual Sheets

In [23]:
# Industrialisation DataFrame
ind = pd.read_excel(xls, 'industrialisation')
ind.head()
ind.shape

(2840, 30)

In [66]:
# CO2 dataframe
co2 = pd.read_excel(xls, 'Co2')
co2

,Entity,Code,Year,Annual CO2 emissions (tonnes )
0,Afghanistan,AFG,1949,14656.00
1,Afghanistan,AFG,1950,84272.00
2,Afghanistan,AFG,1951,91600.00
3,Afghanistan,AFG,1952,91600.00
4,Afghanistan,AFG,1953,106256.00
...,...,...,...,...
20848,Zimbabwe,ZWE,2013,11536239.29
20849,Zimbabwe,ZWE,2014,11866348.41
20850,Zimbabwe,ZWE,2015,10907603.94
20851,Zimbabwe,ZWE,2016,9932649.88


In [21]:
co2.shape

(20853, 4)

In [25]:
# Global Temps Dataframe
gltemp = pd.read_excel(xls, 'GlobalLandTemperaturesByCity')
gltemp.head()
gltemp.shape

(1048575, 7)

## Cleaning the DFs

### Cleaning Global Temps By City

In [26]:
gltemp.isna().sum()

dt                                   0
AverageTemperature               47547
AverageTemperatureUncertainty    47547
City                                 0
Country                              0
Latitude                             0
Longitude                            0
dtype: int64

In [29]:
gltemp = gltemp.dropna()

In [30]:
gltemp.isna().sum()

dt                               0
AverageTemperature               0
AverageTemperatureUncertainty    0
City                             0
Country                          0
Latitude                         0
Longitude                        0
dtype: int64

In [31]:
gltemp

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Ã…rhus,Denmark,57.05N,10.33E
5,1744-04-01,5.788,3.624,Ã…rhus,Denmark,57.05N,10.33E
6,1744-05-01,10.644,1.283,Ã…rhus,Denmark,57.05N,10.33E
7,1744-06-01,14.051,1.347,Ã…rhus,Denmark,57.05N,10.33E
8,1744-07-01,16.082,1.396,Ã…rhus,Denmark,57.05N,10.33E
...,...,...,...,...,...,...,...
1048570,1784-04-01,9.541,7.904,Bordeaux,France,44.20N,0.00W
1048571,1784-05-01,17.364,2.295,Bordeaux,France,44.20N,0.00W
1048572,1784-06-01,19.406,2.543,Bordeaux,France,44.20N,0.00W
1048573,1784-07-01,21.318,4.901,Bordeaux,France,44.20N,0.00W


In [35]:
gltemp = gltemp.reset_index()

In [38]:
del gltemp['index']

In [42]:
gltemp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Ã…rhus,Denmark,57.05N,10.33E
1,1744-04-01,5.788,3.624,Ã…rhus,Denmark,57.05N,10.33E
2,1744-05-01,10.644,1.283,Ã…rhus,Denmark,57.05N,10.33E
3,1744-06-01,14.051,1.347,Ã…rhus,Denmark,57.05N,10.33E
4,1744-07-01,16.082,1.396,Ã…rhus,Denmark,57.05N,10.33E


In [48]:
gltemp['dt'] = pd.to_datetime(gltemp['dt'], type='datetime64[ns]')

TypeError: to_datetime() got an unexpected keyword argument 'type'

In [43]:
gltemp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Ã…rhus,Denmark,57.05N,10.33E
1,1744-04-01,5.788,3.624,Ã…rhus,Denmark,57.05N,10.33E
2,1744-05-01,10.644,1.283,Ã…rhus,Denmark,57.05N,10.33E
3,1744-06-01,14.051,1.347,Ã…rhus,Denmark,57.05N,10.33E
4,1744-07-01,16.082,1.396,Ã…rhus,Denmark,57.05N,10.33E


In [60]:
gltemp = gltemp[gltemp['dt'] > pd.Timestamp(year=1999, day=31, month=12)]

In [61]:
gltemp.head()
gltemp.shape

(68817, 7)

In [67]:
gltemp = gltemp.reset_index(drop=True)

In [84]:
one = gltemp.groupby('City').aggregate(np.mean)
one

,AverageTemperature,AverageTemperatureUncertainty
City,,
A CoruÃ±a,14.074411,0.399951
Aachen,10.077699,0.221595
Aalborg,8.947031,0.286319
Aba,27.365540,0.497840
Abadan,26.504540,0.495920
...,...,...
ÃœrÃ¼mqi,7.403270,0.626982
Ã–skemen,2.771368,0.501147
Ã‡orlu,14.518528,0.317963
